In [ ]:
1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
df_to_caf = pd.read_csv(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/CTS_2023.csv",
    encoding="windows-1250",
    sep=";",
)

In [ ]:
df_from_caf = pd.read_csv(
    "/home/thomas/Nextcloud/CodeursEnLiberte/EMS/mobilite/CTS_calcul_anonym_decode.csv",
    encoding="windows-1250",
    sep=";",
)

In [ ]:
df_to_caf.shape

In [ ]:
df_from_caf

In [ ]:
df_from_caf.shape

In [ ]:
resume_data = pd.DataFrame(
    {
        "NB_ABO": df_to_caf.groupby("SERVICE").NOM.count(),
        "NB_CAF": df_from_caf.groupby("SERVICE").LIB_SERVICE.count(),
        "SERVICE_EMS": df_from_caf.groupby("SERVICE").LIB_SERVICE.first(),
    }
).reset_index()
resume_data["NB_MANQUANT"] = resume_data.NB_ABO - resume_data.NB_CAF
resume_data

In [ ]:
resume_data.to_excel("resume_data.xlsx")

In [ ]:
resume_data.NB_ABO.sum() - 1894 - 29717

In [ ]:
resume_data.NB_CAF.sum() - 18386 - 447

In [ ]:
resume_data.NB_MANQUANT.sum() - 1447 - 11331

In [ ]:
tdf = pd.DataFrame({"LIB": df_from_caf.LIB_SERVICE.unique()})
tdf["ajustement_mensuel_num"] = np.where(tdf.LIB.str.contains("ANN"), 10 / 12, 1)
ttdf = tdf.join(
    tdf.LIB.str.split("_", expand=True).rename(
        columns={0: "AGE_BASE", 1: "PÉRIODE", 2: "TARIF"}
    )
)
age_df = pd.DataFrame(
    data={
        "key": [
            "4-17",
            "18-25",
            "26-64",
            "65 et +",
            "+ 65",
            "PMR",
            "ANNUEL  P+R",
            "EMERAUDE",
        ],
        "AGE": [
            "4<=AGE<=17",
            "18<=AGE<=25",
            "26<=AGE<=64",
            "65<=AGE",
            "65<=AGE",
            "AGE_PMR",
            "AGE",  # P+R
            "AGE_EMERAUDE",
        ],
    }
)
tttdf = ttdf.merge(age_df, left_on="AGE_BASE", right_on="key")
assert len(tttdf) == len(tdf)
qf_df = pd.DataFrame(
    {
        "key": ["QF1", " QF1", "QF2", "QF3", "TP", "GRATUIT", None],
        "QF": [
            "0<=QF<=355",
            "0<=QF<=355",
            "355<QF<=560",
            "560<QF<=765",
            "765<QF",
            "QF_AGE418",
            "QF_EMERAUDE",
        ],
    }
)
meta_df = tttdf.merge(qf_df, left_on="TARIF", right_on="key")  # , how="outer")
meta_df["exclu"] = meta_df.LIB.str.contains("P\+R") + meta_df.LIB.str.contains(
    "GRATUIT"
)
assert len(meta_df) == len(tdf)

In [ ]:
qf_df

In [ ]:
meta_df

In [ ]:
df = df_from_caf

In [ ]:
df3 = df.merge(meta_df, left_on="LIB_SERVICE", right_on="LIB")

In [ ]:
df3

In [ ]:
from mobilite import build_data, compute_result, set_ajustement_mensuel_num

In [ ]:
df3["quantité"] = 1
df3["Titres"] = df3.LIB
df3["idx"] = range(len(df3))
dff = df3[~df3.exclu]
rdf_base = pd.DataFrame(
    data={
        "pu_fichier": dff.ajustement_mensuel_num,
        "pu_calc": dff.ajustement_mensuel_num * 0,
        "tp": dff.ajustement_mensuel_num * 0,
    }
)

In [ ]:
dff.shape

In [ ]:
data_base, complement_df_base = build_data(dff, rdf_base)

In [ ]:
dff

In [ ]:
data_base["input_data_frame_by_entity"]["famille"]

In [ ]:
data_base["input_data_frame_by_entity"]["famille"].shape

In [ ]:
from openfisca_france import CountryTaxBenefitSystem

In [ ]:
from scenario import StrasbourgSurveyScenario

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
compens_constant = 0

In [ ]:
scenario_base = StrasbourgSurveyScenario(base, data=data_base)

In [ ]:
res_base = compute_result(scenario_base, complement_df_base, 0, compens_constant)

In [ ]:
45929

In [ ]:
res_base[0].shape

In [ ]:
res_base[1]

In [ ]:
df_qf_caf = pd.DataFrame(
    {
        "idx": range(len(dff)),
        "QF": "QF==" + dff["MONTANT.QF.CNAF"].astype("str"),
        "AGE": dff.AGE,
        "ajustement_mensuel_num": dff.ajustement_mensuel_num,
        "quantité": 1,
        "Titres": dff.LIB,
    }
)
rdf_qf_caf = pd.DataFrame(
    data={
        "pu_fichier": df_qf_caf.ajustement_mensuel_num,
        "pu_calc": df_qf_caf.ajustement_mensuel_num * 0,
        "tp": df_qf_caf.ajustement_mensuel_num * 0,
    }
)

In [ ]:
data_qf_caf, complement_df_qf_caf = build_data(df_qf_caf, rdf_qf_caf)

In [ ]:
scenario_qf_caf = StrasbourgSurveyScenario(base, data=data_qf_caf)

In [ ]:
res_qf_caf = compute_result(scenario_qf_caf, complement_df_qf_caf, 0, compens_constant)

In [ ]:
res_qf_caf[0].shape

In [ ]:
res_qf_caf[1]

In [ ]:
comp_base_caf = res_base[0].merge(
    res_qf_caf[0], on="individu_id", suffixes=["_base", "_caf"]
)

In [ ]:
comp_base_caf["QF_CAF"] = data_qf_caf["input_data_frame_by_entity"][
    "famille"
].qf_caf.values

In [ ]:
comp_base_caf["QF_EMS"] = data_qf_caf["input_data_frame_by_entity"][
    "famille"
].qf_fiscal.values

In [ ]:
comp_base_caf["QF_RULE"] = dff.QF.values

In [ ]:
comp_base_caf[
    comp_base_caf.titre_fichier_base.str.contains("_TP")
    * (~comp_base_caf.QF_RULE.isna())
].QF_RULE.unique()

In [ ]:
aout23 = "2023-08"

In [ ]:
scenario_qf_caf_aout = StrasbourgSurveyScenario(base, data=data_qf_caf, period=aout23)

In [ ]:
res_qf_caf_aout = compute_result(
    scenario_qf_caf_aout, complement_df_qf_caf, 0, compens_constant, aout23
)

In [ ]:
res_qf_caf_aout[1]

In [ ]:
comp_base_caf["pu_calc_aout23"] = res_qf_caf_aout[0].pu_calc.values

In [ ]:
clean_comp = comp_base_caf[
    [
        "titre_fichier_base",
        "QF_RULE",
        "pu_calc_base",
        "QF_CAF",
        "pu_calc_caf",
        "pu_calc_aout23",
        "QF_EMS",
    ]
]

In [ ]:
clean_comp

In [ ]:
# clean_comp.to_excel('resultats_correspondances_caf_v2.xlsx')

In [ ]:
clean_comp[clean_comp.titre_fichier_base.str.contains("_TP")]